In [51]:
import pandas as pd


In [52]:
#import the findings
df = pd.read_csv("../hars_general_news_out.csv")
for csv in ["../hars_entertainment_out.csv", "../hars_online_shopping_out.csv", "../hars_sports_out.csv", "../hars_sports_out.csv", "../hars_games_out.csv", "../hars_travel_out.csv"]: 
    df = pd.concat([df,pd.read_csv(csv)],ignore_index=True)
    
# for csv in ["../hars_entertainment_out.csv", "../hars_online_shopping_out.csv", "../hars_sports_out.csv", "../hars_sports_out.csv", "../"]
# remove location related leakages 
dropIndex = df[ (df["plain_pii"]=="60201") | (df["plain_pii"]=="Illinois") | (df["plain_pii"]=="Evanston") | (df["third_party"].isna())].index
df.drop(labels=dropIndex,axis=0,inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()


,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,leak_method,encoding,plain_pii,raw_pii,where,key,value,mimeType,text,index
0,2,time.com,https://time.com/create-account-login,api.rlcdn.com,GET,2023-02-18T04:54:05.844Z,get,md5,rogernadal322@gmail.com,NaN,value,iv,b47b636ac3fcb606572fe960f38b703f,NaN,NaN,0.0
1,3,time.com,https://time.com/create-account-login,api.rlcdn.com,GET,2023-02-18T04:54:05.844Z,get,sha256,rogernadal322@gmail.com,NaN,value,iv,eba23eb9253d668bc24763ce39c4a3b51c5cdd8fd25b1c...,NaN,NaN,0.0
2,4,time.com,https://time.com/create-account-login,api.rlcdn.com,GET,2023-02-18T04:54:05.844Z,get,sha1,rogernadal322@gmail.com,NaN,value,iv,b4607c2cd5ad9662de56c4605e5c1d165e949332,NaN,NaN,0.0
3,5,time.com,https://time.com/create-account-login,api.rlcdn.com,GET,2023-02-18T04:54:05.844Z,header,md5,rogernadal322@gmail.com,NaN,value,:path,/api/identity/envelope?pid=13444&it=4&iv=eba23...,NaN,NaN,163.0
4,6,time.com,https://time.com/create-account-login,api.rlcdn.com,GET,2023-02-18T04:54:05.844Z,header,sha256,rogernadal322@gmail.com,NaN,value,:path,/api/identity/envelope?pid=13444&it=4&iv=eba23...,NaN,NaN,41.0


In [53]:
df.groupby(df["first_party"]).count()

,Unnamed: 0,pageref,third_party,http_method,startedDateTime,leak_method,encoding,plain_pii,raw_pii,where,key,value,mimeType,text,index
first_party,,,,,,,,,,,,,,,
airbnb.com,40,8,40,40,40,40,40,40,0,40,40,40,0,0,40
aliexpress.com,49,49,49,49,49,49,49,49,0,49,43,43,6,6,49
allmusic.com,3,3,3,3,3,3,3,3,0,3,0,0,3,3,3
arlo.com,34,34,34,34,34,34,34,34,0,34,22,22,12,12,34
as.com,2,2,2,2,2,2,2,2,0,2,0,0,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wowhead.com,2,2,2,2,2,2,2,2,0,2,2,2,0,0,2
wsj.com,8,8,8,8,8,8,8,8,0,8,6,6,2,2,8
wyndhamhotels.com,10,10,10,10,10,10,10,10,0,10,8,8,2,2,10


In [54]:
#third parties simplified to etld+1
from publicsuffixlist import PublicSuffixList
psl = PublicSuffixList()
# we only want to see the first party to same third party once
df.loc[:,"third_party"] = df.loc[:,"third_party"].map(lambda tp : psl.privatesuffix(tp))

nodups = df.drop_duplicates(subset=["first_party", "third_party"])
# nodups = df
# remove instances of customized subdomain
nodups.loc[:,"third_party"] = nodups.loc[:,"third_party"].map(lambda x : psl.privatesuffix(x)) 


/tmp/ipykernel_110826/3170461779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodups.loc[:,"third_party"] = nodups.loc[:,"third_party"].map(lambda x : psl.privatesuffix(x))


In [55]:
# how many first parties have this third party
nodups["third_party"].groupby(nodups["third_party"]).count().sort_values(ascending=False).to_csv("partners.csv")
nodups["third_party"].groupby(nodups["third_party"]).count().sort_values(ascending=False)
# nodups["third_party"].groupby(nodups["third_party"]).count().sort_values(ascending=False).aggregate('sum', axis=0)


third_party
facebook.com        49
snapchat.com        17
rlcdn.com           13
pinterest.com       12
tiktok.com          11
                    ..
intercom.io          1
inside-graph.com     1
infolinks.com        1
igodigital.com       1
zerobounce.net       1
Name: third_party, Length: 140, dtype: int64

In [56]:
#number of third parties per first party
nodups["first_party"].groupby(nodups["first_party"]).count().sort_values(ascending=False).to_csv("partners.csv")
nodups["first_party"].groupby(nodups["first_party"]).count().sort_values(ascending=False)
# nodups["first_party"].groupby(nodups["first_party"]).count().sort_values(ascending=False).aggregate('sum', axis=0)

first_party
polygon.com           11
gamer.com.tw           9
hindustantimes.com     9
viki.com               8
chewy.com              7
                      ..
callofduty.com         1
shein.com              1
skroutz.gr             1
marca.com              1
getyourguide.com       1
Name: first_party, Length: 138, dtype: int64

In [57]:
nodups.loc[nodups['first_party'] == "hindustantimes.com"]

,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,leak_method,encoding,plain_pii,raw_pii,where,key,value,mimeType,text,index
406,1135,hindustantimes.com,https://accounts.hindustantimes.com/?htMoe=09b...,livemint.com,GET,2023-02-25T09:57:28.017Z,get,sha256,rogernadal322@gmail.com,NaN,value,ud[em],eba23eb9253d668bc24763ce39c4a3b51c5cdd8fd25b1c...,NaN,NaN,0.0
408,1137,hindustantimes.com,https://accounts.hindustantimes.com/?htMoe=09b...,facebook.com,GET,2023-02-25T09:57:34.354Z,get,sha256,rogernadal322@gmail.com,NaN,value,udff[em],eba23eb9253d668bc24763ce39c4a3b51c5cdd8fd25b1c...,NaN,NaN,0.0
416,1145,hindustantimes.com,https://images.livemint.com/ssoconnect/login.h...,webengage.com,GET,2023-02-25T09:58:02.822Z,get,plaintext,rogernadal322,NaN,value,lp,https://images.livemint.com/ssoconnect/login.h...,NaN,NaN,142.0
422,1151,hindustantimes.com,https://images.livemint.com/ssoconnect/login.h...,livehindustan.com,GET,2023-02-25T09:58:04.088Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0
437,1166,hindustantimes.com,https://images1.livehindustan.com/ssoconnect/l...,healthshots.com,GET,2023-02-25T09:58:05.482Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0
441,1170,hindustantimes.com,https://images.ottplay.com/ssoconnect/login.ht...,ottplay.com,GET,2023-02-25T09:58:06.464Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0
443,1172,hindustantimes.com,https://images.slurrp.com/ssoconnect/login.htm...,slurrp.com,GET,2023-02-25T09:58:07.501Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0
445,1174,hindustantimes.com,https://www.htsmartcast.com/ssoconnect/login.h...,htsmartcast.com,GET,2023-02-25T09:58:07.878Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0
447,1176,hindustantimes.com,https://images.htreaders.com/ssoconnect/login....,htreaders.com,GET,2023-02-25T09:58:08.228Z,get,plaintext,rogernadal322,NaN,value,name,rogernadal322,NaN,NaN,0.0


In [58]:
#number of first parties per third party
nodups.loc[nodups['first_party'] == "wired.com"]

,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,leak_method,encoding,plain_pii,raw_pii,where,key,value,mimeType,text,index
455,1217,wired.com,https://id.condenast.com/interaction/QkxhTSH3s...,condenast.com,POST,2023-02-18T05:37:12.016Z,post,plaintext,rogernadal322@gmail.com,NaN,body,NaN,NaN,application/x-www-form-urlencoded,oidcSessionInvalidate=&email=rogernadal322%40g...,29.0
495,1268,wired.com,https://www.wired.com/?redirectURL=/,pippio.com,GET,2023-02-18T05:46:00.115Z,get,md5,rogernadal322@gmail.com,NaN,value,iv,b47b636ac3fcb606572fe960f38b703f,NaN,NaN,0.0
501,1274,wired.com,https://www.wired.com/?redirectURL=/,rlcdn.com,GET,2023-02-18T05:46:00.643Z,get,md5,rogernadal322@gmail.com,NaN,value,m,b47b636ac3fcb606572fe960f38b703f,NaN,NaN,0.0


In [59]:
# piis leaked
df.groupby(df["plain_pii"]).count()


,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,leak_method,encoding,raw_pii,where,key,value,mimeType,text,index
plain_pii,,,,,,,,,,,,,,,
+18729858970,4,4,4,4,4,4,4,4,0,4,0,0,4,4,4
123Andy#!000,60,60,55,60,60,60,60,60,0,60,0,0,60,60,60
Male,3,3,3,3,3,3,3,3,0,3,0,0,3,3,3
nadal,291,291,277,291,291,291,291,291,0,291,230,230,61,61,291
roger,372,372,353,372,372,372,372,372,0,372,289,289,83,83,372
rogernadal,92,92,92,92,92,92,92,92,0,92,89,89,3,3,92
rogernadal322,448,448,187,448,448,448,448,448,0,448,369,369,79,79,448
rogernadal322@gmail.com,1621,1621,1531,1621,1621,1621,1621,1621,0,1621,1244,1244,377,377,1621


In [60]:
#leak method
df.groupby(df["leak_method"]).count()


,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,encoding,plain_pii,raw_pii,where,key,value,mimeType,text,index
leak_method,,,,,,,,,,,,,,,
cookie,119,119,119,119,119,119,119,119,0,119,119,119,0,0,119
get,764,764,719,764,764,764,764,764,0,764,764,764,0,0,764
header,1338,1338,1033,1338,1338,1338,1338,1338,0,1338,1338,1338,0,0,1338
post,670,670,631,670,670,670,670,670,0,670,0,0,670,670,670


In [61]:
# which keys were used in http headers
df[df["leak_method"]=="header"]

,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,leak_method,encoding,plain_pii,raw_pii,where,key,value,mimeType,text,index
3,5,time.com,https://time.com/create-account-login,rlcdn.com,GET,2023-02-18T04:54:05.844Z,header,md5,rogernadal322@gmail.com,NaN,value,:path,/api/identity/envelope?pid=13444&it=4&iv=eba23...,NaN,NaN,163.0
4,6,time.com,https://time.com/create-account-login,rlcdn.com,GET,2023-02-18T04:54:05.844Z,header,sha256,rogernadal322@gmail.com,NaN,value,:path,/api/identity/envelope?pid=13444&it=4&iv=eba23...,NaN,NaN,41.0
5,7,time.com,https://time.com/create-account-login,rlcdn.com,GET,2023-02-18T04:54:05.844Z,header,sha1,rogernadal322@gmail.com,NaN,value,:path,/api/identity/envelope?pid=13444&it=4&iv=eba23...,NaN,NaN,114.0
13,22,nypost.com,https://nypost.com/account/login,spot.im,POST,2023-02-18T07:22:11.328Z,header,base64,roger,NaN,value,x-access-token,eyJhbGciOiJIUzI1NiJ9.eyJzdWIiOiJ1X0RnaUNoSGhRc...,NaN,NaN,145.0
14,23,nypost.com,https://nypost.com/account/login,referralsaasquatch.com,POST,2023-02-18T07:22:23.003Z,header,base64,roger,NaN,value,x-saasquatch-user-token,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJ1c2VyI...,NaN,NaN,193.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2880,822,fourseasons.com,https://reservations.fourseasons.com/en/profil...,facebook.com,GET,2023-02-26T07:44:43.447Z,header,sha256,nadal,NaN,value,:path,/tr/?id=1432546213739567&ev=PageView&dl=https:...,NaN,NaN,248.0
2881,823,fourseasons.com,https://reservations.fourseasons.com/en/profil...,facebook.com,GET,2023-02-26T07:44:43.447Z,header,sha256,rogernadal322@gmail.com,NaN,value,:path,/tr/?id=1432546213739567&ev=PageView&dl=https:...,NaN,NaN,396.0
2885,827,fourseasons.com,https://reservations.fourseasons.com/en/profil...,pinterest.com,GET,2023-02-26T07:44:43.457Z,header,sha256,roger,NaN,value,:path,"/v3/?event=PageVisit&ed={""value"":0}&tid=261418...",NaN,NaN,169.0
2886,828,fourseasons.com,https://reservations.fourseasons.com/en/profil...,pinterest.com,GET,2023-02-26T07:44:43.457Z,header,sha256,nadal,NaN,value,:path,"/v3/?event=PageVisit&ed={""value"":0}&tid=261418...",NaN,NaN,297.0


In [62]:
#encodings
df.groupby(df["encoding"]).count()


,Unnamed: 0,first_party,pageref,third_party,http_method,startedDateTime,leak_method,plain_pii,raw_pii,where,key,value,mimeType,text,index
encoding,,,,,,,,,,,,,,,
base64,276,276,270,276,276,276,276,276,0,276,227,227,49,49,276
md5,107,107,107,107,107,107,107,107,0,107,89,89,18,18,107
plaintext,1310,1310,1029,1310,1310,1310,1310,1310,0,1310,896,896,414,414,1310
sha1,58,58,58,58,58,58,58,58,0,58,58,58,0,0,58
sha256,1134,1134,1038,1134,1134,1134,1134,1134,0,1134,947,947,187,187,1134
sha256ofmd5,6,6,0,6,6,6,6,6,0,6,4,4,2,2,6


In [63]:
#find third party receiving same infromation from more than one first party
grouped = df.groupby(["third_party","plain_pii"])["first_party"].nunique().reset_index()
grouped
grouped[grouped["first_party"]>1].sort_values("first_party", ascending=False)
#.to_csv("suspected_leakers.csv")
# df.groupby(["third_party","plain_pii"]).apply(lambda x:x).reset_index()

,third_party,plain_pii,first_party
53,facebook.com,rogernadal322@gmail.com,47
205,snapchat.com,rogernadal322@gmail.com,17
50,facebook.com,roger,17
49,facebook.com,nadal,14
184,rlcdn.com,rogernadal322@gmail.com,13
165,pinterest.com,rogernadal322@gmail.com,12
219,tiktok.com,rogernadal322@gmail.com,11
30,criteo.com,rogernadal322@gmail.com,7
78,google-analytics.com,rogernadal322@gmail.com,6
88,gravatar.com,rogernadal322@gmail.com,5


In [64]:
grouped = df.groupby(["third_party","plain_pii"])["first_party"].apply(set).reset_index()
grouped


,third_party,plain_pii,first_party
0,adara.com,rogernadal322@gmail.com,{ostrovok.ru}
1,ajax.googleapis.com,rogernadal,{gamer.com.tw}
2,ajax.googleapis.com,rogernadal322@gmail.com,{gamer.com.tw}
3,akstat.io,rogernadal322@gmail.com,{chewy.com}
4,aliexpress.us,123Andy#!000,{aliexpress.com}
...,...,...,...
262,yandex.com,rogernadal322@gmail.com,{rbc.ru}
263,yandex.ru,roger,{thebyrut.org}
264,yandex.ru,rogernadal322,{thebyrut.org}
265,yieldoptimizer.com,rogernadal322@gmail.com,{marriott.com}


In [65]:
#find the favored id per third party
